# Questioning Barbie and Oppenheimer Through the Use of Agents

In the following notebook we will build an application that queries both the Barbie and Oppenheimer movies Wikipedia pages, as well as their reviews. 

The main focus of this notebook is to showcase a brief introduction to Agents.

## Build 🏗️

There are 3 main tasks in this notebook:

1. Contruct a Barbie retriever
2. Construct an Oppenheimer retriever
3. Combine the two and allow users to query both resources from a single input through the use of Agents

## Ship 🚢

Based on Tuesday's session - construct a Chainlit (or Gradio) application that allows users to interface with the application.

## Share 🚀

Make a social media post about your final application.

### Dependencies

As always, let's start with some dependencies!

In [1]:
!pip install -q -U langchain openai

In [1]:
import getpass
import os
openai_api_key = getpass.getpass("Enter your OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

### LLM 

We will be leveraging OpenAI's `gpt-3.5-turbo` throughout the notebook, and we can keep it consistent throughout!

In [2]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature = 0)

### Data Collection and Transformation

We'll be leveraging the `WikipediaLoader` tool to collect information from Wikipedia. 

Be sure to set the `doc_content_chars_max` parameter so that you capture the *entire* Wikipedia article content.

In [6]:
!pip install -q -U wikipedia

In [3]:
from langchain.document_loaders import WikipediaLoader, CSVLoader

barbie_wikipedia_docs = WikipediaLoader(
    query="Barbie (film)", 
    load_max_docs= 1, # YOUR CODE HERE, 
    doc_content_chars_max=10000000 ### YOUR CODE HERE
    ).load()

barbie_csv_docs = CSVLoader(
    file_path= "./barbie_data/barbie.csv", ### YOUR CODE HERE, 
    source_column="Review"   ### YOUR CODE HERE
    ).load()

Since we'll be using same format source documentation separated by topic, we can save ourselves some extra effort and set up our splitters once. 

We're going to leverage the `RecursiveCharacterTextSplitter` again, this time paying close attention to the format our Wikipedia articles and reviews are in so we can be sure to chunk them appropritately. 

> HINT: You can pass a list of separators when you intialize your `RecursiveTextSplitter`! They are acted on in order of element 0 -> element len(list).

RELEVANT DOCS:
- [`RecursiveCharacterTextSplitter`](https://api.python.langchain.com/en/latest/text_splitter/langchain.text_splitter.RecursiveCharacterTextSplitter.html#langchain.text_splitter.RecursiveCharacterTextSplitter)

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

wikipedia_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1024,  ### YOUR CODE HERE,
    chunk_overlap = 512, ### YOUR CODE HERE,
    length_function = len, ### YOUR CODE HERE,
    is_separator_regex= False,
    separators = ["\n==", "\n", " "] ### YOUR CODE HERE # keep headings, then paragraphs, then sentences
)

csv_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1024, ### YOUR CODE HERE,
    chunk_overlap = 512, ### YOUR CODE HERE,
    length_function = len, ### YOUR CODE HERE,
    is_separator_regex= False,
    separators = ["\n", " "] ### YOUR CODE HERE # keep paragraphs, then sentences
)

# type(barbie_wikipedia_docs[0])
chunked_barbie_wikipedia_docs = wikipedia_text_splitter.transform_documents(barbie_wikipedia_docs) ### YOUR CODE HERE
chunked_barbie_csv_docs = csv_text_splitter.transform_documents(barbie_csv_docs) ### YOUR CODE HERE

#### Retrieval and Embedding Strategy

We've already discussed the useful application of `CacheBackedEmbeddings`, so let's do it again!

RELEVANT DOCS:
- [`CacheBackedEmbeddings`](https://api.python.langchain.com/en/latest/embeddings/langchain.embeddings.cache.CacheBackedEmbeddings.html#langchain-embeddings-cache-cachebackedembeddings)

In [5]:
!pip install -q -U rank_bm25 tiktoken faiss-cpu

In [14]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

# set up cached embeddings store
store = LocalFileStore("./.cache/") ### YOUR CODE HERE

core_embeddings_model = OpenAIEmbeddings() ### YOUR CODE HERE

embedder = CacheBackedEmbeddings.from_bytes_store(core_embeddings_model, store, namespace=core_embeddings_model.model) ### YOUR CODE HERE

We'll implement a `FAISS` vectorstore, and create a retriever from it.

In [15]:
barbie_csv_faiss_retriever = FAISS.from_documents(chunked_barbie_csv_docs, embedder) ### YOUR CODE HERE


There are a number of excellent options to retrieve documents - we'll be looking at an additional example today, which is called the `EnsembleRetriever`.

The method this is using is outlined in [this paper](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf).

The brief explanation is:

1. We collect results from two different retrieval methods over the same corpus
2. We apply a reranking algorithm to rerank our source documents to be the *most relevant* without losing specific or potentially low-ranked information rich documents
3. We feed the top-k results into the LLM with our query as context.

> HINT: Your weight list should be of type `List[float]` and the `sum(List[float])` should be `1`.

In [16]:
# set up BM25 retriever
barbie_wikipedia_bm25_retriever = BM25Retriever.from_documents(
    chunked_barbie_wikipedia_docs ### YOUR CODE HERE
)
barbie_wikipedia_bm25_retriever.k = 1

# set up FAISS vector store
barbie_wikipedia_faiss_store = FAISS.from_documents(
    chunked_barbie_wikipedia_docs, ### YOUR CODE HERE,
    embedder ### YOUR CODE HERE
)
barbie_wikipedia_faiss_retriever = barbie_wikipedia_faiss_store.as_retriever(search_kwargs={"k": 1})

# set up ensemble retriever
barbie_ensemble_retriever = EnsembleRetriever(
    retrievers=[barbie_wikipedia_bm25_retriever, barbie_wikipedia_faiss_retriever], ### YOUR CODE HERE,
    weights= [0.25, 0.75] ### YOUR CODE HERE # should sum to 1
)

#### Retrieval Agent

We can create a simple conversational retrieval Agent by using the built-ins provided by LangChain!

> HINT: Be sure to provide good natural language descriptions of what the tool should be used for to get the best results.

RELEVANT DOCS:
- [`create_retriever_tool`](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool.html#langchain.agents.agent_toolkits.conversational_retrieval.tool.create_retriever_tool)

In [17]:
from langchain.agents.agent_toolkits import create_retriever_tool

barbie_wikipedia_retrieval_tool = create_retriever_tool(
    retriever=barbie_ensemble_retriever, ### YOUR CODE HERE 
    name='Search_Wikipedia', ### YOUR CODE HERE
    description='Useful for when you need to answer questions about plot, cast, production, release, music, marketing, reception, themes and analysis of the Barbie movie.' ### YOUR CODE HERE
)

barbie_csv_retrieval_tool = create_retriever_tool(
    retriever=barbie_csv_faiss_retriever.as_retriever(), ### YOUR CODE HERE 
    name='Search_Reviews', ### YOUR CODE HERE
    description='Useful for when you need to answer questions about public reviews of the Barbie movie.'
)

barbie_retriever_tools = [barbie_wikipedia_retrieval_tool, barbie_csv_retrieval_tool] ### YOUR CODE HERE

Now that we've created our tools, we can combined them into an agent!

RELEVANT DOCS:
- [`create_conversational_retrieval_agent`](https://api.python.langchain.com/en/latest/agents/langchain.agents.agent_toolkits.conversational_retrieval.openai_functions.create_conversational_retrieval_agent.html#langchain.agents.agent_toolkits.conversational_retrieval.openai_functions.create_conversational_retrieval_agent)

In [18]:
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

barbie_retriever_agent_executor = create_conversational_retrieval_agent(llm=llm, tools=barbie_retriever_tools, verbose=True) # YOUR CODE HERE

In [11]:
barbie_retriever_agent_executor({"input" : "Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?"})



> Entering new AgentExecutor chain...

Invoking: `Search_Reviews` with `Barbie movie`


[Document(page_content="Review: The big summer blockbuster of 2023 is a wonderful visual spectacle with a great deal of creative physical comedy, centered in and around the universe of Barbie, Mattel's historic toy doll. Unfortunately, the film tries to be and do much more than what the filmmakers ultimately are able to control and reel in. That a movie about Barbie would be guilty of Americanization is no surprise, and in many ways more than befitting, but the Americanization here isn't just about the history of the toy itself and all its cultural impacts - Barbie also views everything through the eyes of Hollywood anno 2023 - which arguably is a very particular and perhaps not that representative outlook. Instead of utilizing the fine foundation laid in the film's first third, director and co-writer Greta Gervig (Lady Bird, Little Women) gets herself lost in a trite, divisive and most likely fut

{'input': 'Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?',
 'chat_history': [HumanMessage(content='Did people like Barbie, or did they find it too Philosphical? If they did, can you tell me why the movie is so Philosophical?', additional_kwargs={}, example=False),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'Search_Reviews', 'arguments': '{\n  "__arg1": "Barbie movie"\n}'}}, example=False),
  FunctionMessage(content='[Document(page_content="Review: The big summer blockbuster of 2023 is a wonderful visual spectacle with a great deal of creative physical comedy, centered in and around the universe of Barbie, Mattel\'s historic toy doll. Unfortunately, the film tries to be and do much more than what the filmmakers ultimately are able to control and reel in. That a movie about Barbie would be guilty of Americanization is no surprise, and in many ways more than befitting, but the America

In [26]:
barbie_retriever_agent_executor({"input" : "What is a very quick summary of the plot of the Barbie movie?"})



> Entering new AgentExecutor chain...

Invoking: `Search_Wikipedia` with `Barbie`


[Document(page_content='== Plot ==\nStereotypical Barbie ("Barbie") and fellow dolls reside in Barbieland; a matriarchal society with different variations of Barbies, Kens, and a group of discontinued models, who are treated like outcasts due to their unconventional traits. While the Kens spend their days playing at the beach, considering it as their profession, the Barbies hold prestigious jobs such as doctors, lawyers, and politicians. Beach Ken ("Ken") is only happy when he is with Barbie and seeks a closer relationship, but Barbie rebuffs him in favor of other activities and female friendships.\nAt a dance party, Barbie is suddenly stricken with worries about mortality. Overnight, she develops bad breath, cellulite, and flat feet, disrupting her usual routines the next day. Weird Barbie, an outcast due to her disfigurement, tells her she must find the child playing with her in the real world to cu

### Oppenheimer Retrieval System

We're going to repourpose some of what we created previously, but this time we'll explore a different multi-source retrieval system.

In [19]:
oppenheimer_wikipedia_docs = WikipediaLoader( 
    query= "Oppenheimer", ### YOUR CODE HERE
    load_max_docs=1, ### YOUR CODE HERE
    doc_content_chars_max=10000000 ### YOUR CODE HERE
).load()

oppenheimer_csv_docs = CSVLoader( 
    file_path= "./oppenheimer_data/oppenheimer.csv", ### YOUR CODE HERE
    source_column="Review" ### YOUR CODE HERE
).load()

In [20]:
chunked_opp_wikipedia_docs = wikipedia_text_splitter.transform_documents(oppenheimer_wikipedia_docs) ### YOUR CODE HERE
chunked_opp_csv_docs = csv_text_splitter.transform_documents(oppenheimer_csv_docs) ### YOUR CODE HERE

In [24]:
opp_csv_faiss_retriever = FAISS.from_documents(chunked_opp_csv_docs, embedder) ### YOUR CODE HERE
opp_csv_faiss_retriever = opp_csv_faiss_retriever.as_retriever() ### MY CODE HERE

# set up BM25 retriever
opp_wikipedia_bm25_retriever = BM25Retriever.from_documents(chunked_opp_wikipedia_docs) ### YOUR CODE HERE
opp_wikipedia_bm25_retriever.k = 1

# set up FAISS vector store
opp_wikipedia_faiss_store = FAISS.from_documents(
    chunked_opp_wikipedia_docs, ### YOUR CODE HERE
    embedder  ### YOUR CODE HERE
) 
opp_wikipedia_faiss_retriever = opp_wikipedia_faiss_store.as_retriever(search_kwargs={"k": 1})

# set up ensemble retriever
opp_ensemble_retriever = EnsembleRetriever(
    retrievers=[opp_wikipedia_bm25_retriever, opp_wikipedia_faiss_retriever], ### YOUR CODE HERE,
    weights= [0.25, 0.75] # should sum to 1 ### YOUR CODE HERE
)

#### Multi-source chain

We're going to allow the LLM to decide which information is most -> least valuable.

The way we'll do this is with LangChain's rather powerful "Expression Language"!

> HINT: You can leverage [this](https://python.langchain.com/docs/use_cases/question_answering/how_to/multiple_retrieval) resource if you get stuck - but experiment with different prompts/formats.

In [ ]:
from langchain.prompts import ChatPromptTemplate

system_message = """Use the information from the below two sources to answer any questions.

Source 1: public user reviews about the Oppenheimer movie
<source1>
{source1}
</source1>

Source 2: the wikipedia page for the Oppenheimer movie including the plot summary, cast, and production information
<source2>
{source2}
</source2>
"""

prompt = ChatPromptTemplate.from_messages([("system", system_message), ("human", "{question}")])

In [ ]:
oppenheimer_multisource_chain = {
    "source1": (lambda x: x["question"]) | opp_ensemble_retriever,
    "source2": (lambda x: x["question"]) | opp_csv_faiss_retriever,
    "question": lambda x: x["question"],
} | prompt | llm

In [ ]:
oppenheimer_multisource_chain.invoke({"question" : "What did people think of the Oppenheimer movie?"})

# Agent Creation

Now we can finally start building our Agent!

The first thing we'll need to do is provide our Agent a Toolbelt. (list of tools). Much like Batman, our LLM-powered Agent can use these tools as it sees fit. 

While the examples we're constructing in this notebook are straightforward for brevity and simplicities sake - there is no limit to what you can build with Agents, as we'll see as we progress through the program.

So, let's begin by setting up our Tools!

You'll notice that we have to set up a function to allow our `OppenheimerInfo` tool to interface with the Agent - this is due to it have a specific required input. Creating custom tools is a pattern that you'll want to grow acustomed to as you use LangChain more and more.

In [ ]:
from langchain.agents import Tool

def query_oppenheimer(input):
    return oppenheimer_multisource_chain.invoke({"question" : input})

tools = [
    Tool(
        name = ### YOUR CODE HERE,
        func=### YOUR CODE HERE,
        description=### YOUR CODE HERE
    ),
    Tool(
        name = ### YOUR CODE HERE,
        func= ### YOUR CODE HERE,
        description= ### YOUR CODE HERE
    ),
]

Now that we've set up our Agents toolbelt, let's set up the LLM that will be powering it!

I would suggest playing around with these prompts - and experiments to find what works best for you.

RELEVANT DOCS:
- [`ZeroShotAgent`](https://api.python.langchain.com/en/latest/agents/langchain.agents.mrkl.base.ZeroShotAgent.html#langchain-agents-mrkl-base-zeroshotagent)

In [ ]:
from langchain.agents import ZeroShotAgent, AgentExecutor

prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    ### YOUR CODE HERE
    ### YOUR CODE HERE
    ### YOUR CODE HERE
    ### YOUR CODE HERE
)

In [ ]:
from langchain import LLMChain

llm_chain = ### YOUR CODE HERE

All that's left to do now is create our `ZeroShotAgent` and our `AgentExecutor`, which are the "reasoner" and "actor" halfs of the `ReAct` method of Agent implementation.

Read all about the `ReAct` framework [here](https://react-lm.github.io/)

In [ ]:
barbenheimer_agent = ZeroShotAgent(
    llm_chain=### YOUR AGENTS, 
    tools=### YOUR AGENTS, 
    verbose=### YOUR AGENTS)

barbenheimer_agent_chain = AgentExecutor.from_agent_and_tools(
    agent=### YOUR AGENTS, 
    tools=### YOUR AGENTS, 
    verbose=### YOUR AGENTS)

## Conclusion

All that is left to do now, is feed inputs to your Agent and watch it work!

Remember to use the `{"input" : "YOUR QUERY HERE"}` format when prompting the Agent.

In [ ]:
barbenheimer_agent_chain.invoke({"input" : "What did people like about the Barbie movie?"})

In [ ]:
barbenheimer_agent_chain.run({"input" : "What did people like about the Oppenheimer movie?"})

In [ ]:
barbenheimer_agent_chain.run({"input" : "Did the movies Barbie and Oppenheimer share similar themes or ideas?"})

## Next Steps

It's time to build a Chainlit (or Gradio) application and host it on Hugging Face Spaces! :ship: